# Secrets
This notebook is used to test code for working with the secrets directory and for generating SAS tokens when necessary. Some of this has or will be moved into the ooiod library in this repo.

## Load keys
Most of this will eventually be moved into a library in this repo.

In [ ]:
import glob, os

def get_keys(secrets_path):
    secrets_path = os.path.abspath(secrets_path)
    if os.path.isdir(secrets_path):
        keys = dict()
        for filename in glob.iglob(secrets_path + '/**', recursive=True):
            if os.path.isfile(filename):
                with open(filename, 'rb') as b:
                    b.seek(1)
                    if b.read(8) != b'GITCRYPT':
                        b.seek(0,0)
                        keys[filename.split('/')[-1]] = b.readline().strip().decode()
        if len(keys) > 0:
            return keys
        else:
            raise Exception('No keys found.')
    else:
        raise NameError('Secrets directory not found.')

In [ ]:
keys = get_keys('../secrets')
keys.keys()

## Generate SAS token
These blocks create a SAS token for a blob container in the ooiopendata storage account.

In [ ]:
from azure.storage.blob import BlockBlobService
from azure.storage.blob.models import ContainerPermissions
import datetime

In [ ]:
ooiopendata_service = BlockBlobService('ooiopendata', keys['ooiopendata_account_key'])

In [ ]:
container_sas = ooiopendata_service.generate_container_shared_access_signature('camhd', 
                ContainerPermissions.READ +
                ContainerPermissions.WRITE +
                ContainerPermissions.DELETE +
                ContainerPermissions.LIST, 
                datetime.datetime.utcnow() + datetime.timedelta(days=1))